# Examples for using the Radial velocity templates by [Braga+ 2021](https://ui.adsabs.harvard.edu/abs/2021ApJ...919...85B/abstract)

This notebook will guide the user in all the possible cases in which it is possible to apply the radial velocity (RV) templates. The aim is always to derive a systemic radial velocity from 

1) A few RV measurements (see details for each case)

2) The knowledge of pulsation properties from optical data (see details for each case)

In [59]:
# Note that RV_template_function imports the module mpfit, 
# which is a third-party product, available within astrolibpy
import RV_template_functions
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PyPDF2 import PdfMerger
import os

First of all, set up the folder in which you will be working

In [49]:
folder = os.getcwd()+'/' #To be changed by the user
folder_coefficient_table = os.getcwd()+'/' #To be changed by the user
folder_rrls = folder_coefficient_table+'test/' #To be changed by the user
folder_rvcs = folder_rrls+'rvcs/'
file_coeff = 'coefficients.csv'

In [50]:
# dictionary for the conversion of pulsation mode into integer
dict_pulsationmode = {'RRab': 0, 'RRc': 1}

The following three cases will be discussed

1) When three or more RV measurements are available. Fe RV-curve templates are used in this case. The template must be used as a fitting function with two free parameters (DeltaPhase, DeltaMag).

2) When four or more RV measurements are available. Fe RV-curve templates are used in this case. The template must be used as a fitting function with three free parameters (DeltaPhase, DeltaMag, Ampl).

## Case 1: Three or more RV measurement available (from Fe lines, with fixed amplitude)

When three or more RV measurements are available for one target, it is possible to use the template not by anchoring it to a given epoch, but as a fitting function, leaving two free parameters: the phase displacement and the mean RV displacement. In this case, only Period, and amplitude are needed as previous knowledge. Note that the function apply_template_templfit_amplfixed takes as input t0, but this can be arbitrary and only affects visualization and not the results.

In [64]:
df = pd.read_csv(folder_rrls+'rrls_properties.csv', dtype={'my_id': str})

# use 0 for generic metallic lines or Fe I multiplet 43
# use 1 for Na doublet
# use 2 for Mg I b triplet
# 3, 4, 5, 6 for Balmer alpha-delta lines
diagnostic = 0
filein = folder_coefficient_table+file_coeff

merger = PdfMerger()

id = []
v_gamma = []
v_gamma_err = []
list_figure_out = []

for ind in df.index:
    
    df_rvc = pd.read_csv(folder_rvcs+df['my_id'][ind]+'.rvc', header=None, delim_whitespace=True)
    n_points = df_rvc.shape[0]
    if n_points < 3:
        continue
    
    df_rvc['HJD'] = df_rvc[4]*1000. + df_rvc[5]
    df_rvc.rename(columns={0: 'RV', 1: 'errRV'}, inplace=True)
    df_rvc.drop(columns=[2, 3, 4, 5, 6, 7, 8, 9, 10], inplace=True)
    
    figureout = folder_rrls + df['my_id'][ind]+'.pdf'
    
    result_case1 = RV_template_functions.apply_template_templfit_amplfixed(df_rvc['HJD'], df_rvc['RV'], 
                                                                           df_rvc['errRV'], df['Vampl'][ind], 
                                                                           dict_pulsationmode[df['class_final'][ind]],
                                                                           df['period_photo'][ind], 0., diagnostic,
                                                                           filein, figure_out=figureout, quiet=1)
    
    id.append(df['my_id'][ind])
    v_gamma.append(result_case1['v_gamma_mean'])
    v_gamma_err.append(result_case1['errv_gamma_mean'])
    list_figure_out.append(figureout)
    
    merger.append(figureout)
    
df_result = pd.DataFrame(data={})
df_result['ID'] = id
df_result['vgamma'] = v_gamma
df_result['v_gamma_err'] = v_gamma_err

merger.write(folder_rrls+'allfigures_case1.pdf')
merger.close()
for pdf in list_figure_out:
    os.remove(pdf)

df_result.to_csv(folder_rrls+'results_case1.csv')

## Case 2: Four or more RV measurement available (from Fe lines, with free amplitude)

When three or more RV measurements are available for one target, it is possible to use the template not by anchoring it to a given epoch, but as a fitting function, leaving three free parameters: the phase displacement, the mean RV displacement and the amplitude ARV. In this case, only Period is needed as previous knowledge. Note that the function apply_template_templfit_amplfixed takes as input t0, but this can be arbitrary and only affects visualization and not the results.

In [63]:
df = pd.read_csv(folder_rrls+'rrls_properties.csv', dtype={'my_id': str})

# use 0 for generic metallic lines or Fe I multiplet 43
# use 1 for Na doublet
# use 2 for Mg I b triplet
# 3, 4, 5, 6 for Balmer alpha-delta lines
diagnostic = 0
filein = folder_coefficient_table+file_coeff

merger = PdfMerger()

id = []
v_gamma = []
v_gamma_err = []
list_figure_out = []

for ind in df.index:
    
    df_rvc = pd.read_csv(folder_rvcs+df['my_id'][ind]+'.rvc', header=None, delim_whitespace=True)
    n_points = df_rvc.shape[0]
    print(n_points)
    if n_points < 4:
        continue

    df_rvc['HJD'] = df_rvc[4]*1000. + df_rvc[5]
    df_rvc.rename(columns={0: 'RV', 1: 'errRV'}, inplace=True)
    df_rvc.drop(columns=[2, 3, 4, 5, 6, 7, 8, 9, 10], inplace=True)
    
    figureout = folder_rrls + df['my_id'][ind]+'.pdf'
    
    result_case1 = RV_template_functions.apply_template_templfit_amplfree(df_rvc['HJD'], df_rvc['RV'], 
                                                                           df_rvc['errRV'], 
                                                                           dict_pulsationmode[df['class_final'][ind]],
                                                                           df['period_photo'][ind], 0., diagnostic,
                                                                           filein, figure_out=figureout, quiet=1)
    
    id.append(df['my_id'][ind])
    v_gamma.append(result_case1['v_gamma_mean'])
    v_gamma_err.append(result_case1['errv_gamma_mean'])
    list_figure_out.append(figureout)
    
    merger.append(figureout)
    
df_result = pd.DataFrame(data={})
df_result['ID'] = id
df_result['vgamma'] = v_gamma
df_result['v_gamma_err'] = v_gamma_err

merger.write(folder_rrls+'allfigures_case2.pdf')
merger.close()
for pdf in list_figure_out:
    os.remove(pdf)

df_result.to_csv(folder_rrls+'results_case2.csv')

4
6
3
6
6
